MongoDB Actualiza Inventario General

In [43]:
input_db = input('Ingresa el nombre del Database del proyecto: ')

In [44]:
from pymongo import MongoClient
import pandas as pd

client = MongoClient('mongodb://localhost:27018/')

def mongoelt(df, database, collection_name):
    db = client[database]
    collection = db[collection_name]
    
    for i, row in df.iterrows():
        item = row['Item']
        phase = row['Phase']
        to_item = row['TakeOff Item']
        unit_converter = row['Unit Converter']
        
        # Check if the document already exists
        document = collection.find_one({'Item': item})
        if document is None:
            if pd.isna(to_item):
                collection.insert_one({'Item': item, 'Phases': [phase]})
            else:
                # If the document does not exist, insert a new one
                collection.insert_one({'Item': item, 'Phases': [phase], 'TakeOff Items': {to_item: {'Unit Converter': unit_converter}}})
        else:
            # If the document exists, check if the phase already exists in the 'Phases' field
            phases = document.get('Phases', [])
            if phase not in phases:
                phases.append(phase)
                collection.update_one({'Item': item}, {'$set': {'Phases': phases}})
            
            # Check if the TakeOff item already exists in the 'TakeOff Items' field
            takeoff_items = document.get('TakeOff Items', {})
            if not pd.isna(to_item) and to_item not in takeoff_items.keys():
                takeoff_items[to_item] = {'Unit Converter': unit_converter}
                collection.update_one({'Item': item}, {'$set': {'TakeOff Items': takeoff_items}})
                                 
dataframe = pd.read_excel("dbfeeder.xlsx", sheet_name='maincatalog')
dataframe[['Item','TakeOff Item']] = dataframe[['Item','TakeOff Item']].applymap(lambda x: x.lower() if isinstance(x, str) else x)
dataframe = dataframe.reset_index()

mongoelt(dataframe,'main_reference','main_reference_table')

MongoDB actualiza DB por Proyecto

In [45]:
from pymongo import MongoClient
import pandas as pd

client = MongoClient('mongodb://localhost:27018/')

def mongoelt(df, database, collection_name):
    db = client[database]
    collection = db[collection_name]
    
    for i, row in df.iterrows():
        item = row['Item']
        phase = row['Phase']
        to_item = row['TakeOff Item']
        unit_converter = row['Unit Converter']
        
        # Check if the document already exists
        document = collection.find_one({'Item': item})
        if document is None:
            if pd.isna(to_item):
                collection.insert_one({'Item': item, 'Phases': [phase]})
            else:
                # If the document does not exist, insert a new one
                collection.insert_one({'Item': item, 'Phases': [phase], 'TakeOff Items': {to_item: {'Unit Converter': unit_converter}}})
        else:
            # If the document exists, check if the phase already exists in the 'Phases' field
            phases = document.get('Phases', [])
            if phase not in phases:
                phases.append(phase)
                collection.update_one({'Item': item}, {'$set': {'Phases': phases}})
            
            # Check if the TakeOff item already exists in the 'TakeOff Items' field
            takeoff_items = document.get('TakeOff Items', {})
            if not pd.isna(to_item) and to_item not in takeoff_items.keys():
                takeoff_items[to_item] = {'Unit Converter': unit_converter}
                collection.update_one({'Item': item}, {'$set': {'TakeOff Items': takeoff_items}})
                    
                
dataframe = pd.read_excel("dbfeeder.xlsx", sheet_name='proyectcatalog')
dataframe[['Item','TakeOff Item']] = dataframe[['Item','TakeOff Item']].applymap(lambda x: x.lower() if isinstance(x, str) else x)
dataframe = dataframe.reset_index()

mongoelt(dataframe, input_db, 'main_reference_table')

Llena el TakeOff Reference

In [46]:
from pymongo import MongoClient
import pandas as pd

client = MongoClient('mongodb://localhost:27018/')

def mongoelt(df, database, collection_name):
    db = client[database]
    collection = db[collection_name]
    
    for i, row in df.iterrows():
        item = row['Item']
        to_item = row['TakeOff Item']
        unit_converter = row['Unit Converter']
        # Check if the document already exists
        document = collection.find_one({'Item': item})
        if document is None:
            collection.insert_one({'Item': item, 'TakeOff Items': {to_item: {'Unit Converter': unit_converter}}})
        else:
            collection.update_one({'Item': item}, {'$set': {'TakeOff Items.' + to_item + '.Unit Converter': unit_converter}})
                    
                
dataframe = pd.read_excel("dbfeeder.xlsx", sheet_name='takeoff_reference')
dataframe = dataframe.applymap(lambda x: x.lower() if isinstance(x, str) else x)
dataframe = dataframe.reset_index()

mongoelt(dataframe, input_db, 'takeoff_reference')

Alimenta Grouping table general

In [47]:
from pymongo import MongoClient
import pandas as pd
client = MongoClient('mongodb://localhost:27018/')
def mongoelt(df, database, collection_name):
    db = client[database]
    collection = db[collection_name]
    
    for i, row in df.iterrows():
        grouping_item = row['Grouping Item']
        item = row['Item']
        unit_converter = row['Unit Converter']
        
        # Check if the document already exists
        document = collection.find_one({'Grouping Item': grouping_item})
        if document is None:
            # Document doesn't exist, insert a new one
            collection.insert_one({'Grouping Item': grouping_item, 'Items': {item: {'Unit Converter': unit_converter}}})
        else:
            # Document already exists
            items = document['Items']
            
            # Check if the item already exists
            if item in items:
                # Item already exists, update the unit converter
                items[item]['Unit Converter'] = unit_converter
            else:
                # Item doesn't exist, add a new item
                items[item] = {'Unit Converter': unit_converter}
            
            # Update the document with the modified items
            collection.update_one(
                {'Grouping Item': grouping_item},
                {'$set': {'Items': items}}
            )
                    
                
dataframe = pd.read_excel("dbfeeder.xlsx", sheet_name='grouping_table')
dataframe = dataframe.applymap(lambda x: x.lower() if isinstance(x, str) else x)
dataframe = dataframe.reset_index()

mongoelt(dataframe, 'main_reference', 'grouping_table')


Alimenta Grouping table del proyecto

In [48]:
from pymongo import MongoClient
import pandas as pd
client = MongoClient('mongodb://localhost:27018/')
def mongoelt(df, database, collection_name):
    db = client[database]
    collection = db[collection_name]
    
    for i, row in df.iterrows():
        grouping_item = row['Grouping Item']
        item = row['Item']
        unit_converter = row['Unit Converter']
        
        # Check if the document already exists
        document = collection.find_one({'Grouping Item': grouping_item})
        if document is None:
            # Document doesn't exist, insert a new one
            collection.insert_one({'Grouping Item': grouping_item, 'Items': {item: {'Unit Converter': unit_converter}}})
        else:
            # Document already exists
            items = document['Items']
            
            # Check if the item already exists
            if item in items:\
                # Item already exists, update the unit converter
                items[item]['Unit Converter'] = unit_converter
            else:
                # Item doesn't exist, add a new item
                items[item] = {'Unit Converter': unit_converter}
            
            # Update the document with the modified items
            collection.update_one(
                {'Grouping Item': grouping_item},
                {'$set': {'Items': items}}
            )
                    
                
dataframe = pd.read_excel("dbfeeder.xlsx", sheet_name='proyect_grouping_table')
dataframe = dataframe.applymap(lambda x: x.lower() if isinstance(x, str) else x)
dataframe = dataframe.reset_index()

mongoelt(dataframe, input_db, 'grouping_table')